In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.activations import linear, relu

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy import sqrt

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [3]:
train_df.head(10)

,id,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,target
0,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
1,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
2,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
3,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
4,13,A,B,A,A,A,A,A,A,B,...,0.364464,0.401162,0.26847,0.46226,0.50556,0.366788,0.359249,0.345247,0.726792,5142.87
5,14,A,A,A,A,B,A,A,A,A,...,0.381515,0.363768,0.24564,0.40455,0.47225,0.334828,0.352251,0.342239,0.382931,1132.22
6,20,A,B,A,B,A,A,A,A,B,...,0.867021,0.583389,0.90267,0.84847,0.80218,0.644013,0.785706,0.859764,0.242416,3585.75
7,23,A,B,B,B,B,A,A,A,B,...,0.628534,0.384099,0.61229,0.38249,0.51111,0.682315,0.669033,0.756454,0.361191,10280.20
8,24,A,B,A,A,B,B,A,A,B,...,0.713343,0.469223,0.30260,0.67135,0.83510,0.863052,0.879347,0.822493,0.294523,6184.59
9,33,A,B,A,A,B,A,A,A,B,...,0.314683,0.370419,0.58354,0.46226,0.38016,0.644013,0.665644,0.339244,0.799124,5965.73


In [4]:
train_df.tail(10)

,id,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,target
169476,587605,B,A,A,A,A,B,A,A,A,...,0.599275,0.548122,0.48864,0.45391,0.64056,0.592525,0.590961,0.701266,0.362479,1173.30
169477,587606,A,A,A,A,B,A,A,A,A,...,0.201125,0.259395,0.24564,0.30859,0.21983,0.207238,0.204687,0.357400,0.348217,2161.12
169478,587607,A,B,A,B,B,B,A,A,B,...,0.269520,0.338963,0.33906,0.28066,0.30529,0.245410,0.261799,0.181433,0.398571,4080.42
169479,587612,A,A,A,A,B,A,A,A,A,...,0.502705,0.473897,0.43518,0.66201,0.58257,0.415029,0.406090,0.354344,0.377315,994.85
169480,587619,A,A,A,A,A,B,A,A,A,...,0.445008,0.377930,0.36636,0.29095,0.44467,0.327915,0.321570,0.731059,0.721499,804.28
169481,587620,A,B,A,A,A,A,A,A,B,...,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
169482,587624,A,A,A,A,A,B,A,A,A,...,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
169483,587630,A,B,A,A,A,A,A,B,B,...,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
169484,587632,A,B,A,A,A,A,A,A,B,...,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87
169485,587633,B,A,A,B,A,A,A,A,A,...,0.844563,0.533048,0.97123,0.93383,0.83814,0.932195,0.946432,0.810511,0.721460,4751.72


In [5]:
%%time
cat = [f'col_{i}' for i in range(1, 117)]
con = [f'col_{i}' for i in range(117, 131)]
for c in cat:
    train_df[c] = train_df[c].astype('category')
    test_df[c] = test_df[c].astype('category')
    train_df[c] = train_df[c].cat.codes
    test_df[c] = test_df[c].cat.codes
del c
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

Wall time: 12.4 s


In [6]:
train_df.columns

Index(['col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8',
       'col_9', 'col_10',
       ...
       'col_122', 'col_123', 'col_124', 'col_125', 'col_126', 'col_127',
       'col_128', 'col_129', 'col_130', 'target'],
      dtype='object', length=131)

In [7]:
X = train_df.iloc[:,train_df.columns != 'target'].values
y = train_df.iloc[:,train_df.columns == 'target'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
del X, y

In [9]:
X_train.shape

(95335, 130)

In [10]:
activation_fn = 'relu'
model = Sequential()
model.add(Input(shape=(130,)))
model.add(Dense(260, activation=activation_fn))
model.add(Dense(130, activation=activation_fn))
model.add(Dense(65, activation=activation_fn))
model.add(Dense(1, activation=linear))
model.compile(optimizer=RMSprop(learning_rate=1e-3), loss=MeanSquaredError(), metrics=['mse', 'mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 260)               34060     
_________________________________________________________________
dense_1 (Dense)              (None, 130)               33930     
_________________________________________________________________
dense_2 (Dense)              (None, 65)                8515      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 66        
Total params: 76,571
Trainable params: 76,571
Non-trainable params: 0
_________________________________________________________________


In [11]:
mse_history = []
mae_history = []

In [12]:
%%time
init_epochs = 0
epochs_now = 100
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'linear_4/model_{epochs_now}.h5')

Epoch 1/100
2980/2980 [==============================] - 13s 4ms/step - loss: 5424851.5000 - mse: 5424851.5000 - mae: 1459.7018 - val_loss: 5032704.5000 - val_mse: 5032704.5000 - val_mae: 1697.2418
Epoch 2/100
2980/2980 [==============================] - 14s 5ms/step - loss: 4593660.0000 - mse: 4593660.0000 - mae: 1334.7026 - val_loss: 4005077.2500 - val_mse: 4005077.2500 - val_mae: 1316.4584
Epoch 3/100
2980/2980 [==============================] - 12s 4ms/step - loss: 4507337.0000 - mse: 4507337.0000 - mae: 1317.4562 - val_loss: 3945072.7500 - val_mse: 3945072.7500 - val_mae: 1260.7466
Epoch 4/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4454946.0000 - mse: 4454946.0000 - mae: 1306.9351 - val_loss: 3998572.7500 - val_mse: 3998572.7500 - val_mae: 1245.2523
Epoch 5/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4402723.0000 - mse: 4402723.0000 - mae: 1299.6893 - val_loss: 4432255.5000 - val_mse: 4432255.5000 - val_mae: 1260.9688
Epoch 6/100


2980/2980 [==============================] - 9s 3ms/step - loss: 4051342.0000 - mse: 4051342.0000 - mae: 1248.0057 - val_loss: 3850156.5000 - val_mse: 3850156.5000 - val_mae: 1220.4028
Epoch 43/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4043752.2500 - mse: 4043752.2500 - mae: 1245.7251 - val_loss: 3915644.7500 - val_mse: 3915644.7500 - val_mae: 1233.1138
Epoch 44/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4036298.2500 - mse: 4036298.2500 - mae: 1246.8428 - val_loss: 3963083.5000 - val_mse: 3963083.5000 - val_mae: 1216.2889
Epoch 45/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4060466.7500 - mse: 4060466.7500 - mae: 1246.9309 - val_loss: 3753300.7500 - val_mse: 3753300.7500 - val_mae: 1249.5977
Epoch 46/100
2980/2980 [==============================] - 8s 3ms/step - loss: 4049607.5000 - mse: 4049607.5000 - mae: 1246.1123 - val_loss: 3781179.2500 - val_mse: 3781179.2500 - val_mae: 1288.1517
Epoch 47/100
2980/2980 

2980/2980 [==============================] - 9s 3ms/step - loss: 3948235.7500 - mse: 3948235.7500 - mae: 1234.3481 - val_loss: 3757209.7500 - val_mse: 3757209.7500 - val_mae: 1233.2015
Epoch 84/100
2980/2980 [==============================] - 7s 3ms/step - loss: 3944232.2500 - mse: 3944232.2500 - mae: 1234.5404 - val_loss: 3762858.0000 - val_mse: 3762858.0000 - val_mae: 1222.9717
Epoch 85/100
2980/2980 [==============================] - 8s 3ms/step - loss: 3943947.2500 - mse: 3943947.2500 - mae: 1231.7017 - val_loss: 3826620.2500 - val_mse: 3826620.2500 - val_mae: 1219.3035
Epoch 86/100
2980/2980 [==============================] - 9s 3ms/step - loss: 3936719.2500 - mse: 3936719.2500 - mae: 1231.4930 - val_loss: 3778468.5000 - val_mse: 3778468.5000 - val_mae: 1212.2037
Epoch 87/100
2980/2980 [==============================] - 8s 3ms/step - loss: 3950250.2500 - mse: 3950250.2500 - mae: 1232.2272 - val_loss: 4116233.2500 - val_mse: 4116233.2500 - val_mae: 1230.8459
Epoch 88/100
2980/2980 

In [13]:
%%time
init_epochs = 100
epochs_now = 500
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'linear_4/model_{epochs_now}.h5')

Epoch 101/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3915278.2500 - mse: 3915278.2500 - mae: 1229.1018 - val_loss: 3749320.5000 - val_mse: 3749320.5000 - val_mae: 1226.3833
Epoch 102/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3872621.7500 - mse: 3872621.7500 - mae: 1228.9949 - val_loss: 3751294.2500 - val_mse: 3751294.2500 - val_mae: 1264.8848
Epoch 103/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3900788.5000 - mse: 3900788.5000 - mae: 1229.4510 - val_loss: 3967374.7500 - val_mse: 3967374.7500 - val_mae: 1287.9169
Epoch 104/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3890459.2500 - mse: 3890459.2500 - mae: 1228.4453 - val_loss: 3717576.0000 - val_mse: 3717576.0000 - val_mae: 1222.1578
Epoch 105/500
2980/2980 [==============================] - 9s 3ms/step - loss: 3892767.2500 - mse: 3892767.2500 - mae: 1228.7207 - val_loss: 4053567.7500 - val_mse: 4053567.7500 - val_mae: 1378.5249
Epoch

2980/2980 [==============================] - 3s 934us/step - loss: 3803209.5000 - mse: 3803209.5000 - mae: 1219.2833 - val_loss: 3925888.0000 - val_mse: 3925888.0000 - val_mae: 1269.0256
Epoch 143/500
2980/2980 [==============================] - 3s 951us/step - loss: 3824146.0000 - mse: 3824146.0000 - mae: 1219.3866 - val_loss: 4106837.5000 - val_mse: 4106837.5000 - val_mae: 1341.3149
Epoch 144/500
2980/2980 [==============================] - 3s 969us/step - loss: 3798316.7500 - mse: 3798316.7500 - mae: 1218.5997 - val_loss: 3771958.7500 - val_mse: 3771958.7500 - val_mae: 1216.4309
Epoch 145/500
2980/2980 [==============================] - 12s 4ms/step - loss: 3799774.7500 - mse: 3799774.7500 - mae: 1218.5906 - val_loss: 3702869.5000 - val_mse: 3702869.5000 - val_mae: 1248.1348
Epoch 146/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3789478.0000 - mse: 3789478.0000 - mae: 1217.4138 - val_loss: 3733809.0000 - val_mse: 3733809.0000 - val_mae: 1220.6669
Epoch 147/5

2980/2980 [==============================] - 14s 5ms/step - loss: 3727526.5000 - mse: 3727526.5000 - mae: 1207.7883 - val_loss: 3738064.7500 - val_mse: 3738064.7500 - val_mae: 1248.3442
Epoch 184/500
2980/2980 [==============================] - 15s 5ms/step - loss: 3724247.0000 - mse: 3724247.0000 - mae: 1206.2305 - val_loss: 3839479.7500 - val_mse: 3839479.7500 - val_mae: 1264.7943
Epoch 185/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3707744.5000 - mse: 3707744.5000 - mae: 1207.0259 - val_loss: 3780394.0000 - val_mse: 3780394.0000 - val_mae: 1246.9813
Epoch 186/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3724455.5000 - mse: 3724455.5000 - mae: 1204.4885 - val_loss: 3907863.5000 - val_mse: 3907863.5000 - val_mae: 1244.7039
Epoch 187/500
2980/2980 [==============================] - 15s 5ms/step - loss: 3722041.0000 - mse: 3722041.0000 - mae: 1206.6836 - val_loss: 3907777.7500 - val_mse: 3907777.7500 - val_mae: 1228.2965
Epoch 188/500


2980/2980 [==============================] - 13s 4ms/step - loss: 3619888.7500 - mse: 3619888.7500 - mae: 1195.0663 - val_loss: 3905471.0000 - val_mse: 3905471.0000 - val_mae: 1274.5044
Epoch 225/500
2980/2980 [==============================] - 13s 4ms/step - loss: 3625390.5000 - mse: 3625390.5000 - mae: 1196.4236 - val_loss: 3757175.2500 - val_mse: 3757175.2500 - val_mae: 1251.7775
Epoch 226/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3595805.0000 - mse: 3595805.0000 - mae: 1195.1417 - val_loss: 3800780.7500 - val_mse: 3800780.7500 - val_mae: 1265.9481
Epoch 227/500
2980/2980 [==============================] - 10s 3ms/step - loss: 3610222.5000 - mse: 3610222.5000 - mae: 1196.5677 - val_loss: 3858370.7500 - val_mse: 3858370.7500 - val_mae: 1243.6449
Epoch 228/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3601960.0000 - mse: 3601960.0000 - mae: 1194.7374 - val_loss: 3822692.7500 - val_mse: 3822692.7500 - val_mae: 1220.7380
Epoch 229/500


2980/2980 [==============================] - 8s 3ms/step - loss: 3504633.0000 - mse: 3504633.0000 - mae: 1185.9867 - val_loss: 3823308.7500 - val_mse: 3823308.7500 - val_mae: 1305.6689
Epoch 266/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3518545.0000 - mse: 3518545.0000 - mae: 1187.5066 - val_loss: 4181553.7500 - val_mse: 4181553.7500 - val_mae: 1248.4152
Epoch 267/500
2980/2980 [==============================] - 10s 3ms/step - loss: 3522271.0000 - mse: 3522271.0000 - mae: 1185.8031 - val_loss: 3880711.7500 - val_mse: 3880711.7500 - val_mae: 1306.1176
Epoch 268/500
2980/2980 [==============================] - 10s 3ms/step - loss: 3521823.0000 - mse: 3521823.0000 - mae: 1187.8934 - val_loss: 3940493.5000 - val_mse: 3940493.5000 - val_mae: 1232.8627
Epoch 269/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3499761.2500 - mse: 3499761.2500 - mae: 1185.8798 - val_loss: 3888399.2500 - val_mse: 3888399.2500 - val_mae: 1247.9502
Epoch 270/500
29

2980/2980 [==============================] - 10s 3ms/step - loss: 3314582.0000 - mse: 3314582.0000 - mae: 1173.7419 - val_loss: 4153595.7500 - val_mse: 4153595.7500 - val_mae: 1242.4020
Epoch 307/500
2980/2980 [==============================] - 10s 3ms/step - loss: 3375121.7500 - mse: 3375121.7500 - mae: 1175.2992 - val_loss: 4067024.5000 - val_mse: 4067024.5000 - val_mae: 1246.8600
Epoch 308/500
2980/2980 [==============================] - 10s 3ms/step - loss: 3365359.2500 - mse: 3365359.2500 - mae: 1175.2401 - val_loss: 4312128.5000 - val_mse: 4312128.5000 - val_mae: 1295.9806
Epoch 309/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3348854.5000 - mse: 3348854.5000 - mae: 1174.6752 - val_loss: 3923634.5000 - val_mse: 3923634.5000 - val_mae: 1307.2140
Epoch 310/500
2980/2980 [==============================] - 10s 4ms/step - loss: 3375013.0000 - mse: 3375013.0000 - mae: 1174.0278 - val_loss: 4523511.5000 - val_mse: 4523511.5000 - val_mae: 1519.5219
Epoch 311/500


2980/2980 [==============================] - 10s 3ms/step - loss: 3189810.5000 - mse: 3189810.5000 - mae: 1160.1362 - val_loss: 3992614.5000 - val_mse: 3992614.5000 - val_mae: 1261.8302
Epoch 348/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3209664.0000 - mse: 3209664.0000 - mae: 1161.6375 - val_loss: 4108824.5000 - val_mse: 4108824.5000 - val_mae: 1303.4092
Epoch 349/500
2980/2980 [==============================] - 17s 6ms/step - loss: 3212687.0000 - mse: 3212687.0000 - mae: 1157.7048 - val_loss: 3952543.7500 - val_mse: 3952543.7500 - val_mae: 1257.6887
Epoch 350/500
2980/2980 [==============================] - 18s 6ms/step - loss: 3213299.7500 - mse: 3213299.7500 - mae: 1158.7391 - val_loss: 4278780.5000 - val_mse: 4278780.5000 - val_mae: 1347.1980
Epoch 351/500
2980/2980 [==============================] - 17s 6ms/step - loss: 3183853.7500 - mse: 3183853.7500 - mae: 1159.0731 - val_loss: 4016285.2500 - val_mse: 4016285.2500 - val_mae: 1252.9568
Epoch 352/500


2980/2980 [==============================] - 14s 5ms/step - loss: 3098601.0000 - mse: 3098601.0000 - mae: 1149.2242 - val_loss: 4149971.2500 - val_mse: 4149971.2500 - val_mae: 1296.6732
Epoch 389/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3093031.7500 - mse: 3093031.7500 - mae: 1149.4567 - val_loss: 4234080.5000 - val_mse: 4234080.5000 - val_mae: 1395.7371
Epoch 390/500
2980/2980 [==============================] - 9s 3ms/step - loss: 3106477.2500 - mse: 3106477.2500 - mae: 1148.3500 - val_loss: 4177980.7500 - val_mse: 4177980.7500 - val_mae: 1273.4734
Epoch 391/500
2980/2980 [==============================] - 12s 4ms/step - loss: 3117235.0000 - mse: 3117235.0000 - mae: 1147.6910 - val_loss: 4022479.0000 - val_mse: 4022479.0000 - val_mae: 1269.5979
Epoch 392/500
2980/2980 [==============================] - 14s 5ms/step - loss: 3107817.2500 - mse: 3107817.2500 - mae: 1149.0640 - val_loss: 4060456.2500 - val_mse: 4060456.2500 - val_mae: 1305.1956
Epoch 393/500
2

2980/2980 [==============================] - 12s 4ms/step - loss: 3050366.5000 - mse: 3050366.5000 - mae: 1140.5878 - val_loss: 4090093.2500 - val_mse: 4090093.2500 - val_mae: 1273.5472
Epoch 430/500
2980/2980 [==============================] - 8s 3ms/step - loss: 3018814.2500 - mse: 3018814.2500 - mae: 1138.2709 - val_loss: 4394853.5000 - val_mse: 4394853.5000 - val_mae: 1312.8464
Epoch 431/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3028892.2500 - mse: 3028892.2500 - mae: 1139.8523 - val_loss: 4131807.2500 - val_mse: 4131807.2500 - val_mae: 1258.4558
Epoch 432/500
2980/2980 [==============================] - 11s 4ms/step - loss: 3000282.2500 - mse: 3000282.2500 - mae: 1139.8540 - val_loss: 4307211.5000 - val_mse: 4307211.5000 - val_mae: 1377.5054
Epoch 433/500
2980/2980 [==============================] - 12s 4ms/step - loss: 3021728.0000 - mse: 3021728.0000 - mae: 1139.9039 - val_loss: 4350683.0000 - val_mse: 4350683.0000 - val_mae: 1299.2715
Epoch 434/500
2

2980/2980 [==============================] - 10s 3ms/step - loss: 2883666.5000 - mse: 2883666.5000 - mae: 1127.9169 - val_loss: 4250459.5000 - val_mse: 4250459.5000 - val_mae: 1267.6278
Epoch 471/500
2980/2980 [==============================] - 9s 3ms/step - loss: 2908071.2500 - mse: 2908071.2500 - mae: 1126.8267 - val_loss: 4241146.0000 - val_mse: 4241146.0000 - val_mae: 1287.8984
Epoch 472/500
2980/2980 [==============================] - 9s 3ms/step - loss: 2946396.7500 - mse: 2946396.7500 - mae: 1127.8756 - val_loss: 4287681.0000 - val_mse: 4287681.0000 - val_mae: 1291.7058
Epoch 473/500
2980/2980 [==============================] - 12s 4ms/step - loss: 2872383.5000 - mse: 2872383.5000 - mae: 1125.4294 - val_loss: 4174509.5000 - val_mse: 4174509.5000 - val_mae: 1290.9778
Epoch 474/500
2980/2980 [==============================] - 12s 4ms/step - loss: 2895110.0000 - mse: 2895110.0000 - mae: 1124.2469 - val_loss: 4186224.2500 - val_mse: 4186224.2500 - val_mae: 1272.3429
Epoch 475/500
29

In [14]:
%%time
init_epochs = 500
epochs_now = 1000
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'linear_4/model_{epochs_now}.h5')

Epoch 501/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 2878547.5000 - mse: 2878547.5000 - mae: 1120.2710 - val_loss: 4308143.0000 - val_mse: 4308143.0000 - val_mae: 1291.3628
Epoch 502/1000
2980/2980 [==============================] - 14s 5ms/step - loss: 2830495.2500 - mse: 2830495.2500 - mae: 1117.5676 - val_loss: 4222887.0000 - val_mse: 4222887.0000 - val_mae: 1301.7992
Epoch 503/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 2835447.2500 - mse: 2835447.2500 - mae: 1118.9661 - val_loss: 4292343.5000 - val_mse: 4292343.5000 - val_mae: 1334.1627
Epoch 504/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 2864023.0000 - mse: 2864023.0000 - mae: 1118.1766 - val_loss: 4383258.5000 - val_mse: 4383258.5000 - val_mae: 1282.6661
Epoch 505/1000
2980/2980 [==============================] - 14s 5ms/step - loss: 2817572.0000 - mse: 2817572.0000 - mae: 1116.3284 - val_loss: 4428754.0000 - val_mse: 4428754.0000 - val_mae: 1318.

Epoch 542/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 2714440.7500 - mse: 2714440.7500 - mae: 1104.7975 - val_loss: 4494875.5000 - val_mse: 4494875.5000 - val_mae: 1328.9436
Epoch 543/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 2738504.5000 - mse: 2738504.5000 - mae: 1109.5880 - val_loss: 4424317.5000 - val_mse: 4424317.5000 - val_mae: 1340.5115
Epoch 544/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 2737298.5000 - mse: 2737298.5000 - mae: 1106.4646 - val_loss: 4441969.5000 - val_mse: 4441969.5000 - val_mae: 1328.5895
Epoch 545/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 2718192.7500 - mse: 2718192.7500 - mae: 1106.8142 - val_loss: 4479004.0000 - val_mse: 4479004.0000 - val_mae: 1302.2552
Epoch 546/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 2748366.0000 - mse: 2748366.0000 - mae: 1106.8353 - val_loss: 4518138.0000 - val_mse: 4518138.0000 - val_mae: 1316.12

Epoch 583/1000
2980/2980 [==============================] - 12s 4ms/step - loss: 2682808.7500 - mse: 2682808.7500 - mae: 1100.0994 - val_loss: 4631641.5000 - val_mse: 4631641.5000 - val_mae: 1367.3435
Epoch 584/1000
2980/2980 [==============================] - 12s 4ms/step - loss: 2683970.5000 - mse: 2683970.5000 - mae: 1097.3938 - val_loss: 4831459.5000 - val_mse: 4831459.5000 - val_mae: 1357.3813
Epoch 585/1000
2980/2980 [==============================] - 10s 4ms/step - loss: 2661950.5000 - mse: 2661950.5000 - mae: 1096.0463 - val_loss: 4846727.0000 - val_mse: 4846727.0000 - val_mae: 1318.4851
Epoch 586/1000
2980/2980 [==============================] - 13s 5ms/step - loss: 2667126.0000 - mse: 2667126.0000 - mae: 1096.4554 - val_loss: 4586064.0000 - val_mse: 4586064.0000 - val_mae: 1300.4635
Epoch 587/1000
2980/2980 [==============================] - 13s 4ms/step - loss: 2653993.7500 - mse: 2653993.7500 - mae: 1097.9747 - val_loss: 4543537.0000 - val_mse: 4543537.0000 - val_mae: 1301.

Epoch 624/1000
2980/2980 [==============================] - 15s 5ms/step - loss: 2584221.0000 - mse: 2584221.0000 - mae: 1088.1118 - val_loss: 4521513.0000 - val_mse: 4521513.0000 - val_mae: 1320.2968
Epoch 625/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 2607372.2500 - mse: 2607372.2500 - mae: 1088.6575 - val_loss: 4548295.5000 - val_mse: 4548295.5000 - val_mae: 1357.3153
Epoch 626/1000
2980/2980 [==============================] - 10s 3ms/step - loss: 2582587.2500 - mse: 2582587.2500 - mae: 1088.3147 - val_loss: 4613923.5000 - val_mse: 4613923.5000 - val_mae: 1333.3362
Epoch 627/1000
2980/2980 [==============================] - 11s 4ms/step - loss: 2596738.2500 - mse: 2596738.2500 - mae: 1090.3560 - val_loss: 4605581.5000 - val_mse: 4605581.5000 - val_mae: 1324.5275
Epoch 628/1000
2980/2980 [==============================] - 9s 3ms/step - loss: 2585856.5000 - mse: 2585856.5000 - mae: 1088.3137 - val_loss: 5153354.0000 - val_mse: 5153354.0000 - val_mae: 1403.3

Epoch 665/1000
2980/2980 [==============================] - 3s 846us/step - loss: 2559123.0000 - mse: 2559123.0000 - mae: 1082.6967 - val_loss: 4496593.0000 - val_mse: 4496593.0000 - val_mae: 1344.1475
Epoch 666/1000
2980/2980 [==============================] - 2s 836us/step - loss: 2542001.0000 - mse: 2542001.0000 - mae: 1081.0333 - val_loss: 4778501.0000 - val_mse: 4778501.0000 - val_mae: 1359.1254
Epoch 667/1000
2980/2980 [==============================] - 3s 869us/step - loss: 2556320.5000 - mse: 2556320.5000 - mae: 1081.4568 - val_loss: 4728411.5000 - val_mse: 4728411.5000 - val_mae: 1341.6497
Epoch 668/1000
2980/2980 [==============================] - 2s 836us/step - loss: 2533521.7500 - mse: 2533521.7500 - mae: 1080.4460 - val_loss: 4859468.0000 - val_mse: 4859468.0000 - val_mae: 1320.7765
Epoch 669/1000
2980/2980 [==============================] - 3s 855us/step - loss: 2523915.0000 - mse: 2523915.0000 - mae: 1079.7429 - val_loss: 4969967.0000 - val_mse: 4969967.0000 - val_mae: 

...

Epoch 827/1000
2980/2980 [==============================] - 3s 853us/step - loss: 2391455.2500 - mse: 2391455.2500 - mae: 1059.7074 - val_loss: 4767701.0000 - val_mse: 4767701.0000 - val_mae: 1334.6328
Epoch 828/1000
2980/2980 [==============================] - 3s 856us/step - loss: 2386174.0000 - mse: 2386174.0000 - mae: 1061.8499 - val_loss: 5505535.5000 - val_mse: 5505535.5000 - val_mae: 1376.6066
Epoch 829/1000
2980/2980 [==============================] - 2s 834us/step - loss: 2386637.2500 - mse: 2386637.2500 - mae: 1058.3131 - val_loss: 4880159.5000 - val_mse: 4880159.5000 - val_mae: 1339.7017
Epoch 830/1000
2980/2980 [==============================] - 3s 871us/step - loss: 2380121.5000 - mse: 2380121.5000 - mae: 1057.6526 - val_loss: 4990434.5000 - val_mse: 4990434.5000 - val_mae: 1338.9661
Epoch 831/1000
2980/2980 [==============================] - 3s 846us/step - loss: 2393392.5000 - mse: 2393392.5000 - mae: 1057.1852 - val_loss: 4988826.0000 - val_mse: 4988826.0000 - val_mae: 

2980/2980 [==============================] - 3s 846us/step - loss: 2369483.2500 - mse: 2369483.2500 - mae: 1053.8385 - val_loss: 4935936.5000 - val_mse: 4935936.5000 - val_mae: 1348.5009
Epoch 868/1000
2980/2980 [==============================] - 3s 859us/step - loss: 2357679.2500 - mse: 2357679.2500 - mae: 1055.4886 - val_loss: 4940119.0000 - val_mse: 4940119.0000 - val_mae: 1333.5762
Epoch 869/1000
2980/2980 [==============================] - 2s 826us/step - loss: 2377997.7500 - mse: 2377997.7500 - mae: 1057.4448 - val_loss: 5142102.0000 - val_mse: 5142102.0000 - val_mae: 1412.4878
Epoch 870/1000
2980/2980 [==============================] - 3s 847us/step - loss: 2368921.0000 - mse: 2368921.0000 - mae: 1054.8792 - val_loss: 5136911.0000 - val_mse: 5136911.0000 - val_mae: 1358.8514
Epoch 871/1000
2980/2980 [==============================] - 2s 835us/step - loss: 2361089.0000 - mse: 2361089.0000 - mae: 1053.0903 - val_loss: 4824542.0000 - val_mse: 4824542.0000 - val_mae: 1349.9216
Epoch

Epoch 908/1000
2980/2980 [==============================] - 3s 843us/step - loss: 2334353.5000 - mse: 2334353.5000 - mae: 1049.9728 - val_loss: 4912167.0000 - val_mse: 4912167.0000 - val_mae: 1362.8779
Epoch 909/1000
2980/2980 [==============================] - 3s 892us/step - loss: 2334682.5000 - mse: 2334682.5000 - mae: 1048.8615 - val_loss: 4921114.0000 - val_mse: 4921114.0000 - val_mae: 1347.8186
Epoch 910/1000
2980/2980 [==============================] - 3s 854us/step - loss: 2330819.0000 - mse: 2330819.0000 - mae: 1048.4568 - val_loss: 5049617.5000 - val_mse: 5049617.5000 - val_mae: 1343.8025
Epoch 911/1000
2980/2980 [==============================] - 3s 896us/step - loss: 2330313.7500 - mse: 2330313.7500 - mae: 1050.5580 - val_loss: 5156720.0000 - val_mse: 5156720.0000 - val_mae: 1354.3790
Epoch 912/1000
2980/2980 [==============================] - 3s 864us/step - loss: 2326790.5000 - mse: 2326790.5000 - mae: 1049.8590 - val_loss: 5192070.5000 - val_mse: 5192070.5000 - val_mae: 

2980/2980 [==============================] - 3s 864us/step - loss: 2305459.0000 - mse: 2305459.0000 - mae: 1045.9207 - val_loss: 4991748.5000 - val_mse: 4991748.5000 - val_mae: 1346.6587
Epoch 949/1000
2980/2980 [==============================] - 3s 907us/step - loss: 2298636.0000 - mse: 2298636.0000 - mae: 1044.8182 - val_loss: 5090239.5000 - val_mse: 5090239.5000 - val_mae: 1339.8494
Epoch 950/1000
2980/2980 [==============================] - 3s 845us/step - loss: 2320276.5000 - mse: 2320276.5000 - mae: 1045.1904 - val_loss: 5079202.5000 - val_mse: 5079202.5000 - val_mae: 1345.4218
Epoch 951/1000
2980/2980 [==============================] - 3s 849us/step - loss: 2306686.0000 - mse: 2306686.0000 - mae: 1043.7855 - val_loss: 4975293.0000 - val_mse: 4975293.0000 - val_mae: 1369.5446
Epoch 952/1000
2980/2980 [==============================] - 3s 868us/step - loss: 2300070.0000 - mse: 2300070.0000 - mae: 1043.2181 - val_loss: 4998520.5000 - val_mse: 4998520.5000 - val_mae: 1383.7576
Epoch

Epoch 989/1000
2980/2980 [==============================] - 3s 877us/step - loss: 2285610.2500 - mse: 2285610.2500 - mae: 1040.5255 - val_loss: 5262753.0000 - val_mse: 5262753.0000 - val_mae: 1353.0723
Epoch 990/1000
2980/2980 [==============================] - 3s 870us/step - loss: 2297628.2500 - mse: 2297628.2500 - mae: 1043.4683 - val_loss: 5352880.5000 - val_mse: 5352880.5000 - val_mae: 1379.8431
Epoch 991/1000
2980/2980 [==============================] - 3s 876us/step - loss: 2282197.5000 - mse: 2282197.5000 - mae: 1040.4070 - val_loss: 5331217.0000 - val_mse: 5331217.0000 - val_mae: 1369.5492
Epoch 992/1000
2980/2980 [==============================] - 3s 903us/step - loss: 2292338.2500 - mse: 2292338.2500 - mae: 1041.5525 - val_loss: 4938335.0000 - val_mse: 4938335.0000 - val_mae: 1371.0786
Epoch 993/1000
2980/2980 [==============================] - 3s 880us/step - loss: 2281207.5000 - mse: 2281207.5000 - mae: 1039.0643 - val_loss: 5409599.5000 - val_mse: 5409599.5000 - val_mae: 

In [ ]:
%%time
init_epochs = 1000
epochs_now = 5000
history = model.fit(X_train, y_train, epochs=epochs_now, validation_data=(X_val, y_val), initial_epoch=init_epochs)
mse_history += history.history['mse']
mae_history += history.history['mae']
model.save(f'linear_4/model_{epochs_now}.h5')

Epoch 1001/5000
2980/2980 [==============================] - 3s 1ms/step - loss: 2274590.7500 - mse: 2274590.7500 - mae: 1038.8007 - val_loss: 4968783.0000 - val_mse: 4968783.0000 - val_mae: 1394.5133
Epoch 1002/5000
2980/2980 [==============================] - 3s 910us/step - loss: 2284473.7500 - mse: 2284473.7500 - mae: 1040.3524 - val_loss: 5193943.0000 - val_mse: 5193943.0000 - val_mae: 1381.9943
Epoch 1003/5000
2980/2980 [==============================] - 3s 1ms/step - loss: 2279795.5000 - mse: 2279795.5000 - mae: 1039.7544 - val_loss: 5183711.0000 - val_mse: 5183711.0000 - val_mae: 1343.2869
Epoch 1004/5000
2980/2980 [==============================] - 3s 908us/step - loss: 2290142.5000 - mse: 2290142.5000 - mae: 1039.3247 - val_loss: 5208083.5000 - val_mse: 5208083.5000 - val_mae: 1365.1674
Epoch 1005/5000
2980/2980 [==============================] - 3s 929us/step - loss: 2282070.5000 - mse: 2282070.5000 - mae: 1040.9343 - val_loss: 5272417.5000 - val_mse: 5272417.5000 - val_mae:

2980/2980 [==============================] - 3s 1ms/step - loss: 2258450.7500 - mse: 2258450.7500 - mae: 1037.4044 - val_loss: 5135185.0000 - val_mse: 5135185.0000 - val_mae: 1349.1075
Epoch 1042/5000
2980/2980 [==============================] - 3s 907us/step - loss: 2252384.5000 - mse: 2252384.5000 - mae: 1035.7445 - val_loss: 5261184.5000 - val_mse: 5261184.5000 - val_mae: 1362.3148
Epoch 1043/5000
2980/2980 [==============================] - 3s 977us/step - loss: 2252859.0000 - mse: 2252859.0000 - mae: 1034.4827 - val_loss: 5739839.5000 - val_mse: 5739839.5000 - val_mae: 1496.9913
Epoch 1044/5000
2980/2980 [==============================] - 3s 860us/step - loss: 2258304.2500 - mse: 2258304.2500 - mae: 1035.9313 - val_loss: 5318311.5000 - val_mse: 5318311.5000 - val_mae: 1361.7443
Epoch 1045/5000
2980/2980 [==============================] - 3s 917us/step - loss: 2266397.5000 - mse: 2266397.5000 - mae: 1037.1019 - val_loss: 5234968.0000 - val_mse: 5234968.0000 - val_mae: 1419.0638
Epo

2980/2980 [==============================] - 2s 831us/step - loss: 2231582.5000 - mse: 2231582.5000 - mae: 1032.1899 - val_loss: 5233641.0000 - val_mse: 5233641.0000 - val_mae: 1356.7736
Epoch 1082/5000
2980/2980 [==============================] - 3s 844us/step - loss: 2236857.5000 - mse: 2236857.5000 - mae: 1031.3877 - val_loss: 5380667.5000 - val_mse: 5380667.5000 - val_mae: 1376.5479
Epoch 1083/5000
2980/2980 [==============================] - 3s 840us/step - loss: 2233523.2500 - mse: 2233523.2500 - mae: 1031.1956 - val_loss: 5496331.5000 - val_mse: 5496331.5000 - val_mae: 1391.8558
Epoch 1084/5000
2980/2980 [==============================] - 3s 843us/step - loss: 2227084.0000 - mse: 2227084.0000 - mae: 1030.3489 - val_loss: 5394853.0000 - val_mse: 5394853.0000 - val_mae: 1460.0331
Epoch 1085/5000
2980/2980 [==============================] - 2s 836us/step - loss: 2232145.7500 - mse: 2232145.7500 - mae: 1031.1682 - val_loss: 5001165.0000 - val_mse: 5001165.0000 - val_mae: 1353.5867
E

In [ ]:
plt.plot(mae_history)
plt.show()
plt.plot(mse_history)
plt.show()